In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

In [3]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_cc = pd.read_pickle(ENV.credit_card_balance_clean.value)
print('Credit shape: {}'.format(X_cc.shape))

X_cc_fe = pd.read_pickle(ENV.credit_card_balance_clean_rnn.value)
print('Credit shape: {}'.format(X_cc_fe.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Credit shape: (3840312, 23)
Credit shape: (104307, 81)


In [4]:
def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  sum(X_Train.TARGET ==0)/sum(X_Train.TARGET ==1)
class_ratio =  1
def get_rnn_model(num_words,embed_size,embedding_matrix):
    sequence_input = Input(shape=(maxlen, ))
    
    x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(CuDNNGRU(8, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    x = BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
#     x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
#     Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
#     model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)

def train_each_fold(x,y,x_val,y_val,model,filepath,reportpath,predspath,
                    batch_size=512,total_epoch=40,patience=5,saving=True):
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(x,y,batch_size,model)
        y_pred = model.predict(x_val,batch_size=3000,verbose=1)
        score = roc_auc_score(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=512,total_epoch=400,patience=30):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(0,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = model_file.format(fold)
        report_file_evl = report_file.format(fold)
        pred_file_evl = pred_file.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file_evl,reportpath=report_file_evl,predspath=pred_file_evl,
                        batch_size=batch_size,total_epoch=total_epoch,patience=patience)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict(x_test,batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


# Credit Card V1

In [5]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = ['AMT_BALANCE',
             'AMT_CREDIT_LIMIT_ACTUAL',
             'AMT_DRAWINGS_ATM_CURRENT',
             'AMT_DRAWINGS_CURRENT',
             'AMT_DRAWINGS_OTHER_CURRENT',
             'AMT_DRAWINGS_POS_CURRENT',
             'AMT_INST_MIN_REGULARITY',
             'AMT_PAYMENT_CURRENT',
             'AMT_PAYMENT_TOTAL_CURRENT',
             'AMT_RECEIVABLE_PRINCIPAL',
             'AMT_RECIVABLE',
             'AMT_TOTAL_RECEIVABLE',
             'CNT_DRAWINGS_ATM_CURRENT',
             'CNT_DRAWINGS_CURRENT',
             'CNT_DRAWINGS_OTHER_CURRENT',
             'CNT_DRAWINGS_POS_CURRENT',
             'CNT_INSTALMENT_MATURE_CUM',
             'SK_DPD',
             'SK_DPD_DEF']
for col in trans_col:
    print(col)
    X_cc[col] = qt.fit_transform(X_cc[col].values.reshape(-1,1))

AMT_BALANCE
AMT_CREDIT_LIMIT_ACTUAL
AMT_DRAWINGS_ATM_CURRENT
AMT_DRAWINGS_CURRENT
AMT_DRAWINGS_OTHER_CURRENT
AMT_DRAWINGS_POS_CURRENT
AMT_INST_MIN_REGULARITY
AMT_PAYMENT_CURRENT
AMT_PAYMENT_TOTAL_CURRENT
AMT_RECEIVABLE_PRINCIPAL
AMT_RECIVABLE
AMT_TOTAL_RECEIVABLE
CNT_DRAWINGS_ATM_CURRENT
CNT_DRAWINGS_CURRENT
CNT_DRAWINGS_OTHER_CURRENT
CNT_DRAWINGS_POS_CURRENT
CNT_INSTALMENT_MATURE_CUM
SK_DPD
SK_DPD_DEF


In [9]:
print('start')

label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#192
maxlen = 15
# embed_size = 21

sorted_df = X_cc.sort_values(['SK_ID_CURR','MONTHS_BALANCE'])
col = 'MONTHS_BALANCE'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
col = 'MONTHS_BALANCE'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.credit_card_rnn.value
report_file = ENV.credit_card_report.value
pred_file = ENV.credit_card_preds.value
pred_test_file = ENV.credit_card_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30)

...get ebd mat
num of words: 3840314
(3840314, 21)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/creditcard/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 3us/step
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.5483127320646654
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.5483127320646654
current roc is 0.5472155378591802
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 2 / 500
previous best roc is 0.5483127320646654
current roc is 0.5259646429575727
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/creditcard/f

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 23 / 500
previous best roc is 0.5589238045979649
current roc is 0.558740168636048
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/creditcard/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 24 / 500
previous best roc is 0.5589238045979649
current roc is 0.5595639910244418
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 25 / 500
previous best roc is 0.5595639910244418
current roc is 0.5588517426011903
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 26 / 500
previous best roc is 0.5595639910244418

Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 46 / 500
previous best roc is 0.565201164555727
current roc is 0.5539729147895852
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.565201164555727
current roc is 0.5636170165450987
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 48 / 500
previous best roc is 0.565201164555727
current roc is 0.5621530005566682
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 49 / 500
previous best roc is 0.565201164555727
current roc is 0.5634790797470316
Epoch 1/1
61503/61503 [=========================

saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 70 / 500
previous best roc is 0.5657007330660944
current roc is 0.564395351389662
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 71 / 500
previous best roc is 0.5657007330660944
current roc is 0.5648449739886715
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 72 / 500
previous best roc is 0.5657007330660944
current roc is 0.5649098695990267
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 73 / 500
previous best roc is 0.5657007330660944
current roc is 0.564698757108998
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/cred

61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 95 / 500
previous best roc is 0.5669451143752404
current roc is 0.5647464254150623
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 96 / 500
previous best roc is 0.5669451143752404
current roc is 0.5649561103666549
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 97 / 500
previous best roc is 0.5669451143752404
current roc is 0.565298870236039
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 98 / 500
previous best roc is 0.5669451143752404
current roc is 0.5663745634888753
Epoch 1/1
61503/61503 [==============================] 

...

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 71 / 500
previous best roc is 0.562503741108602
current roc is 0.56166823710052
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 72 / 500
previous best roc is 0.562503741108602
current roc is 0.5615965557238649
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 73 / 500
previous best roc is 0.562503741108602
current roc is 0.5618976326944641
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 74 / 500
previous best roc is 0.562503741108602
current roc is 0.5622195795903804
Epoch 1/1
61502/61502 [==============================] - 0s 

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 96 / 500
previous best roc is 0.5629230490449537
current roc is 0.5616289473234259
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 97 / 500
previous best roc is 0.5629230490449537
current roc is 0.5617038978756983
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 98 / 500
previous best roc is 0.5629230490449537
current roc is 0.5628324704590204
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 99 / 500
previous best roc is 0.5629230490449537
current roc is 0.5613593560580385
Epoch 1/1
61502/61502 [=====================

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 120 / 500
previous best roc is 0.5638505926422314
current roc is 0.5620307069470762
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 121 / 500
previous best roc is 0.5638505926422314
current roc is 0.5622908830590134
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 122 / 500
previous best roc is 0.5638505926422314
current roc is 0.5617977167064913
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 123 / 500
previous best roc is 0.5638505926422314
current roc is 0.5633868868418835
Epoch 1/1
61502/61502 [===========================

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 145 / 500
previous best roc is 0.5639084921291994
current roc is 0.5615107326312243
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 146 / 500
previous best roc is 0.5639084921291994
current roc is 0.5630486320395007
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 147 / 500
previous best roc is 0.5639084921291994
current roc is 0.5623979178215464
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 148 / 500
previous best roc is 0.5639084921291994
current roc is 0.5634843400991092
Epoch 1/1
61502/61502 [=================

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 170 / 500
previous best roc is 0.5647281764373502
current roc is 0.5634168265597317
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 171 / 500
previous best roc is 0.5647281764373502
current roc is 0.5636173727637573
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 172 / 500
previous best roc is 0.5647281764373502
current roc is 0.563398958200523
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 173 / 500
previous best roc is 0.5647281764373502
current roc is 0.5608986590564887
Epoch 1/1
61502/61502 [==================

In [7]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean         11.214464
std          24.202830
min           1.000000
25%           1.000000
50%           1.000000
75%           7.000000
max         192.000000
Name: text, dtype: float64

In [8]:
aaaa

NameError: name 'aaaa' is not defined

# Credit Card V2

In [5]:
X_cc_fe = X_cc_fe.rename({'MONTHS_BALANCE_MAX':'CC_MONTHS_BALANCE_MAX',
             'MONTHS_BALANCE_MIN':'CC_MONTHS_BALANCE_MIN',
             'SK_DPD_DEF_MAX':'CC_SK_DPD_DEF_MAX',
             'SK_DPD_DEF_MEAN':'CC_SK_DPD_DEF_MEAN',
             'SK_DPD_DEF_MIN':'CC_SK_DPD_DEF_MIN',
             'SK_DPD_MAX':'CC_SK_DPD_MAX',
             'SK_DPD_MEAN':'CC_SK_DPD_MEAN',
             'SK_DPD_MIN':'CC_SK_DPD_MIN' },axis=1)

In [6]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = [ 'CNT_DRAWINGS_CURRENT_MAX',
       'CNT_DRAWINGS_CURRENT_MIN', 'CNT_DRAWINGS_CURRENT_MEAN',
       'AMT_TOTAL_RECEIVABLE_MAX', 'AMT_TOTAL_RECEIVABLE_MIN',
       'AMT_TOTAL_RECEIVABLE_MEAN', 'AMT_RECIVABLE_MAX', 'AMT_RECIVABLE_MIN',
       'AMT_RECIVABLE_MEAN', 'AMT_RECEIVABLE_PRINCIPAL_MAX',
       'AMT_RECEIVABLE_PRINCIPAL_MIN', 'AMT_RECEIVABLE_PRINCIPAL_MEAN',
       'CC_SK_DPD_MAX', 'CC_SK_DPD_MIN', 'CC_SK_DPD_MEAN',
       'AMT_PAYMENT_TOTAL_CURRENT_MAX', 'AMT_PAYMENT_TOTAL_CURRENT_MIN',
       'AMT_PAYMENT_TOTAL_CURRENT_MEAN', 'AMT_DRAWINGS_CURRENT_MAX',
       'AMT_DRAWINGS_CURRENT_MIN', 'AMT_DRAWINGS_CURRENT_MEAN',
       'AMT_CREDIT_LIMIT_ACTUAL_MAX', 'AMT_CREDIT_LIMIT_ACTUAL_MIN',
       'AMT_CREDIT_LIMIT_ACTUAL_MEAN', 'AMT_BALANCE_MAX', 'AMT_BALANCE_MIN',
       'AMT_BALANCE_MEAN', 'CC_MONTHS_BALANCE_MAX', 'CC_MONTHS_BALANCE_MIN',
       'MONTHS_BALANCE_MEAN', 'CC_SK_DPD_DEF_MAX', 'CC_SK_DPD_DEF_MIN',
       'CC_SK_DPD_DEF_MEAN', 'AMT_INST_MIN_REGULARITY_MAX',
       'AMT_INST_MIN_REGULARITY_MIN', 'AMT_INST_MIN_REGULARITY_MEAN',
       'CNT_INSTALMENT_MATURE_CUM_MAX', 'CNT_INSTALMENT_MATURE_CUM_MIN',
       'CNT_INSTALMENT_MATURE_CUM_MEAN', 'AMT_DRAWINGS_POS_CURRENT_MAX',
       'AMT_DRAWINGS_POS_CURRENT_MIN', 'AMT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_ATM_CURRENT_MAX', 'AMT_DRAWINGS_ATM_CURRENT_MIN',
       'AMT_DRAWINGS_ATM_CURRENT_MEAN', 'CNT_DRAWINGS_ATM_CURRENT_MAX',
       'CNT_DRAWINGS_ATM_CURRENT_MIN', 'CNT_DRAWINGS_ATM_CURRENT_MEAN',
       'CNT_DRAWINGS_OTHER_CURRENT_MAX', 'CNT_DRAWINGS_OTHER_CURRENT_MIN',
       'CNT_DRAWINGS_OTHER_CURRENT_MEAN', 'CNT_DRAWINGS_POS_CURRENT_MAX',
       'CNT_DRAWINGS_POS_CURRENT_MIN', 'CNT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_OTHER_CURRENT_MAX', 'AMT_DRAWINGS_OTHER_CURRENT_MIN',
       'AMT_DRAWINGS_OTHER_CURRENT_MEAN', 'AMT_PAYMENT_CURRENT_MAX',
       'AMT_PAYMENT_CURRENT_MIN', 'AMT_PAYMENT_CURRENT_MEAN', 'Records_CNT',
       'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Completed',
       'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Signed',
       'NAME_CONTRACT_STATUS_Sent_proposal', 'NAME_CONTRACT_STATUS_Refused',
       'NAME_CONTRACT_STATUS_Approved']
for col in trans_col:
    print(col)
    X_cc_fe[col] = qt.fit_transform(X_cc_fe[col].values.reshape(-1,1))

CNT_DRAWINGS_CURRENT_MAX
CNT_DRAWINGS_CURRENT_MIN
CNT_DRAWINGS_CURRENT_MEAN
AMT_TOTAL_RECEIVABLE_MAX
AMT_TOTAL_RECEIVABLE_MIN
AMT_TOTAL_RECEIVABLE_MEAN
AMT_RECIVABLE_MAX
AMT_RECIVABLE_MIN
AMT_RECIVABLE_MEAN
AMT_RECEIVABLE_PRINCIPAL_MAX
AMT_RECEIVABLE_PRINCIPAL_MIN
AMT_RECEIVABLE_PRINCIPAL_MEAN
CC_SK_DPD_MAX
CC_SK_DPD_MIN
CC_SK_DPD_MEAN
AMT_PAYMENT_TOTAL_CURRENT_MAX
AMT_PAYMENT_TOTAL_CURRENT_MIN
AMT_PAYMENT_TOTAL_CURRENT_MEAN
AMT_DRAWINGS_CURRENT_MAX
AMT_DRAWINGS_CURRENT_MIN
AMT_DRAWINGS_CURRENT_MEAN
AMT_CREDIT_LIMIT_ACTUAL_MAX
AMT_CREDIT_LIMIT_ACTUAL_MIN
AMT_CREDIT_LIMIT_ACTUAL_MEAN
AMT_BALANCE_MAX
AMT_BALANCE_MIN
AMT_BALANCE_MEAN
CC_MONTHS_BALANCE_MAX
CC_MONTHS_BALANCE_MIN
MONTHS_BALANCE_MEAN
CC_SK_DPD_DEF_MAX
CC_SK_DPD_DEF_MIN
CC_SK_DPD_DEF_MEAN
AMT_INST_MIN_REGULARITY_MAX
AMT_INST_MIN_REGULARITY_MIN
AMT_INST_MIN_REGULARITY_MEAN
CNT_INSTALMENT_MATURE_CUM_MAX
CNT_INSTALMENT_MATURE_CUM_MIN
CNT_INSTALMENT_MATURE_CUM_MEAN
AMT_DRAWINGS_POS_CURRENT_MAX
AMT_DRAWINGS_POS_CURRENT_MIN
AMT_DRAW

In [8]:

print('start')
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#295
maxlen = 5

sorted_df = X_cc_fe.sort_values(['SK_ID_CURR','CC_MONTHS_BALANCE_MIN'])
col = 'CC_MONTHS_BALANCE_MIN'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.credit_card_rnn.value
report_file = ENV.credit_card_report.value
pred_file = ENV.credit_card_preds.value
pred_test_file = ENV.credit_card_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=35)

start
start normalize
get embedding
ebd size is 79
create document
get embedding Mat
...get ebd mat
num of words: 104309
(104309, 79)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/creditcard/fold_{}.hdf5
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.5508614925452365
Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 1 / 500
previous best roc is 0.5508614925452365
current roc is 0.547360626696547
Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving model to... ../LIB/../../data/rnn/creditcard/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 2 / 500
previous best roc is 0.55086149254

Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving model to... ../LIB/../../data/rnn/creditcard/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 22 / 500
previous best roc is 0.5622930212739095
current roc is 0.5625194403475446
Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving model to... ../LIB/../../data/rnn/creditcard/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 23 / 500
previous best roc is 0.5625194403475446
current roc is 0.56344097020964
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 24 / 500
previous best roc is 0.56344097020964
current roc is 0.5579967152268853
Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/repor

61503/61503 [==============================] - 0s 1us/step
saving model to... ../LIB/../../data/rnn/creditcard/fold_0.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 45 / 500
previous best roc is 0.565810802377031
current roc is 0.5658155458935673
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 46 / 500
previous best roc is 0.5658155458935673
current roc is 0.565403423079351
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 47 / 500
previous best roc is 0.5658155458935673
current roc is 0.5642659923759406
Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 48 / 500
previous best roc is 0.5658155458935673
current ro

61503/61503 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 69 / 500
previous best roc is 0.5666399941750693
current roc is 0.5651188228475016
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 70 / 500
previous best roc is 0.5666399941750693
current roc is 0.5651194756414598
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 71 / 500
previous best roc is 0.5666399941750693
current roc is 0.5651402843455704
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 72 / 500
previous best roc is 0.5666399941750693
current roc is 0.5638047629568841
Epoch 1/1
61503/61503 [==============================]

61503/61503 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 94 / 500
previous best roc is 0.5666399941750693
current roc is 0.5626280623927552
Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 95 / 500
previous best roc is 0.5666399941750693
current roc is 0.5642967902623227
Epoch 1/1
61503/61503 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 96 / 500
previous best roc is 0.5666399941750693
current roc is 0.5641561077841735
Epoch 1/1
61503/61503 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_0.pkl
======= current 97 / 500
previous best roc is 0.5666399941750693
current roc is 0.5626588262047277
Epoch 1/1
61503/61503 [==============================]

...

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_3.pkl
======= current 91 / 500
previous best roc is 0.5668459091223149
current roc is 0.5632261070948921
Epoch 1/1
61502/61502 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_3.pkl
======= current 92 / 500
previous best roc is 0.5668459091223149
current roc is 0.5645832781654879
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_3.pkl
======= current 93 / 500
previous best roc is 0.5668459091223149
current roc is 0.5644967141091337
reach patience! end loop
48744/48744 [==============================] - 0s 3us/step


!!!!!!!! Begin fold: 4
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/creditcard/fold_{}.hdf5
Epoch 1/1
61502/61502 [==============================] - 0s 5us/step
saving preds..

61502/61502 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 19 / 500
previous best roc is 0.5625973357260228
current roc is 0.5603799522694574
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 20 / 500
previous best roc is 0.5625973357260228
current roc is 0.5616614365642942
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 21 / 500
previous best roc is 0.5625973357260228
current roc is 0.5613266498364877
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 22 / 500
previous best roc is 0.5625973357260228
current roc is 0.561959906150358
Epoch 1/1
61502/61502 [==============================] 

61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 43 / 500
previous best roc is 0.5630623488349831
current roc is 0.5609917980144763
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 44 / 500
previous best roc is 0.5630623488349831
current roc is 0.5617313025359053
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 45 / 500
previous best roc is 0.5630623488349831
current roc is 0.5627309234995836
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 46 / 500
previous best roc is 0.5630623488349831
current roc is 0.5623144543938285
Epoch 1/1
61502/61502 [==============================]

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/creditcard/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 68 / 500
previous best roc is 0.5639282825756209
current roc is 0.5644861236369237
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 69 / 500
previous best roc is 0.5644861236369237
current roc is 0.5637602057225676
Epoch 1/1
61502/61502 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 70 / 500
previous best roc is 0.5644861236369237
current roc is 0.5630107273224303
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 71 / 500
previous best roc is 0.564486123636923

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 93 / 500
previous best roc is 0.5644861236369237
current roc is 0.5612618214330567
Epoch 1/1
61502/61502 [==============================] - 0s 1us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 94 / 500
previous best roc is 0.5644861236369237
current roc is 0.5612522471604389
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 95 / 500
previous best roc is 0.5644861236369237
current roc is 0.5627835449314483
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/creditcard/report_fold_4.pkl
======= current 96 / 500
previous best roc is 0.5644861236369237
current roc is 0.5594864782556859
Epoch 1/1
61502/61502 [=====================

In [21]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean          1.001779
std           0.042599
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           4.000000
Name: text, dtype: float64

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()